## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [ ]:
from dotenv import load_dotenv
from langchain.chains.llm import LLMChain
from sympy.physics.units import temperature

load_dotenv()

## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

> - 왜 질문이라고 하지 않고 Prompt로 하는가? 물어보는 용도가 아니라 llm에게 지시한다는 의미이다.
> - ChatGPT에서는 완성된 문장으로 물어보지만, 우리가 만드는 것은 특정 템플릿에 내용을 변경하여 요청하는 구조를 가진다. 즉, 변수를 지정하여 질문할 때 치환하는 구조를 가진다.

In [ ]:
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [ ]:
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

In [ ]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

In [ ]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [ ]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 100자로 설명해주세요.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 예전 사용 방식
# from langchain import LLMChain
# chain = LLMChain(prompt=prompt, llm=model)
chain = prompt | model



### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [ ]:
input = {"topic": "python의 장점"}

In [ ]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

아래는 스트리밍을 출력하는 예시 입니다.

In [ ]:
from share.messages import stream_response

answer = chain.stream(input)
stream_response(answer)

In [ ]:
# 변수가 1개인 경우는 dict으로 전달하지 않아도 된다.
answer = chain.invoke("한국의 수도")
print(answer)

### 출력파서(Output Parser)


In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [ ]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [ ]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [ ]:
template = """
당신은 5년차 스펙트라 개발자입니다. 주어진 질문에 대한 기능을 각 언어에 맞게 작성해주세요.
작성한 언어는 아래의 [FORMAT]을 참고하세요.

#기능:
{question}

#FORMAT:
- Java:
- Python:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [ ]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "1부터 10까지 더하는 함수"}))

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "예외처리하는 방법"})
# 스트리밍 출력
stream_response(answer)